In [1]:
# pip install -r ../requirements.txt

In [2]:
from utils import utils
from app import run_app, App
from core.gen_topics import gen_topics
from graphneo4j import GraphNeo4j
import warnings
warnings.filterwarnings("ignore") # supress all the warnigns




/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Generate topics
  - Use this section to generate the csv's for topics and keywords

In [3]:
settings = utils.get_settings_notebook('../settings_topics.json')
app = App(settings)
dfs = app.extract_transform()

movie_topics = gen_topics()
# (df_movie_topics, df_topic_words) = movie_topics.batch_process(data_df['movies'][data_df['movies']['lang'] == 'en'])
(df_movie_topics, df_topic_words) = movie_topics.batch_process(dfs['movies'][
    dfs['movies']['overview'].apply(lambda x: x is not None and len(x) > 30)
])
df_topic_words.reset_index(drop=True, inplace=True)
df_movie_topics.reset_index(drop=True, inplace=True)
# display(df_topic_words)
# display(df_movie_topics)
df_topic_words.to_csv('./files/archive/topic_words.csv', index=True)
df_movie_topics.to_csv('./files/archive/movie_topic_ids.csv', index=True)

==== TRANSFORM directors ====
==== TRANSFORM movies ====
==== TRANSFORM cast ====


2022-12-11 22:46:41 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-11 22:46:41,866 : INFO : Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-11 22:46:43 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-12-11 22:46:43,151 : INFO : Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
|

## Load the dataframes into graph database (Neo4j)

In [4]:
settings = utils.get_settings_notebook('../settings.json')
# utils.pretty_print(settings)
app = App(settings)
app.etl()

=========  Crwaler award_won ========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   award_company  598 non-null    object
 1   award_type     598 non-null    object
 2   movie          597 non-null    object
 3   person         597 non-null    object
 4   year           598 non-null    object
dtypes: object(5)
memory usage: 23.5+ KB
None
========= end ========
=========  Crwaler award_nominated ========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2832 entries, 0 to 2831
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   award_company  2832 non-null   object
 1   award_type     2832 non-null   object
 2   movie          2816 non-null   object
 3   person         2243 non-null   object
 4   year           2832 non-null   object
dtypes: object(5)
memory usage: 110.8

## ADD KEYWORDS NODE

In [5]:
graph4j = GraphNeo4j(**{**settings['neo4j'], 'clear_data': False})

In [6]:
qry = '''
    MATCH (t:Topic)<-[]-(m:Movie) UNWIND(t.topics) as keyword
    MERGE (k:Keyword {keyword: keyword})
    MERGE (k)<-[:HAS_KEYWORD]-(m)
'''
graph4j.run_query(qry, {})

MATCH (t:Topic)<-[]-(m:Movie) UNWIND(t.topics) as keyword
    MERGE (k:Keyword {keyword: keyword})
    MERGE (k)<-[:HAS_KEYWORD]-(m)
